In [8]:
#########################################
# Chargement des bibliothèques          #
#########################################
%matplotlib inline
import pandas
import numpy
import pylab
import os
import json

path ="/home/user/Documents/NFE204-BDD/Etude OrientDB"
os.chdir(path)

from pymongo import MongoClient
# pprint library is used to make the output look more pretty
from pprint import pprint
from pandas import DataFrame

In [9]:
################################################
# Chargement des données brutes depuis MongoDB #
################################################

# connect to MongoDB
client = MongoClient('localhost:27017')
db=client.airbnb

rec = db.listings.find({"_id" : {'$lte':5000} },
{
"id" : 1, #Id de l'appartement
#"listing_url" : str, #url
#"scrape_id" : int,# inutile car on prend qu'un fichier
#"last_scraped" : str,# inutile car on prend qu'un fichier
"name" : 1, # nome de l'appartement
#"summary" : str, #à récupérer pour analyse textuelle
#"space" : str, #à récupérer pour analyse textuelle
#"description" : str, #à récupérer pour analyse textuelle
#"experiences_offered" : str, #toujours à None
#"neighborhood_overview" : str,#à récupérer pour analyse textuelle NB il existe des descriptions identiques
#"notes" : str,#à récupérer pour analyse textuelle
#transit" : str, # desritpif du Pour y aller ...
#"access" : str,#à récupérer pour analyse textuelle
#"interaction" : str,#à récupérer pour analyse textuelle NB il existe des descriptions identiques
#"house_rules" : str,#à récupérer pour analyse textuelle NB il existe des descriptions identiques
#"thumbnail_url" : str,#url
#"medium_url" : str,#url
#"picture_url" : str,#url
#"xl_picture_url" : str,#url
"host_id" : 1, #int,
#"host_url" : 1, #str,#url
"host_name" : 1, #str,
"host_since" : 1, #str,
"host_location" : 1, #str,
#"host_about" : 1, #str,#à récupérer pour analyse textuelle
"host_response_time" : 1, #str,
"host_response_rate" : 1, #str,
#"host_acceptance_rate" : 1, #str,#non renseigné
"host_is_superhost" : 1, #str,
#"host_thumbnail_url" : str,
#"host_picture_url" : str,
"host_neighbourhood" : 1, #str,
"host_listings_count" : 1, #int,
#"host_total_listings_count" : int,
#"host_verifications" : 1, #str,
#"host_has_profile_pic" : str,
#"host_identity_verified" : 1, #str,
#"street" : 1, #str, #mal renseigné
#"neighbourhood" : 1, #str,
"neighbourhood_cleansed" : 1, #str,
#"neighbourhood_group_cleansed" : 1, #str,#non renseigné
#"city" : 1, #str,
#"state" : 1, #str,
"zipcode" : 1, #str, #à nettoyer
#"market" : 1, #str, #inutile
#"smart_location" : 1, #str, #mal renseigné
#"country_code" : 1, #str,
#"country" : 1, #str,
"latitude" : 1, #float,
"longitude" : 1, #float,
#"is_location_exact" : 1, #str, #inutile
"property_type" : 1, # 95% Appartements - intéressant pour voir si les autres sont atypiques
"room_type" : 1, # 85% où appartement complet, sinon chambre privée ou partagée
"accommodates" : 1, #entre 1 et 16 pièce => recodage en qual ?
"bathrooms" : 1, #entre 0 et 8 mais Q4 toujours à 1 => recodage en qual 0,1,plusieurs
"bedrooms" : 1, #entre 0 et 10 mais Q4 toujours à 1 => recodage en qual 0,1,plusieurs
"beds" : 1, #entre 1 et 16 mais Q4 toujours à 2 => recodage en qual 1,2,plus de 2
#"bed_type" : 1, #regroupement des valeurs faibles
#    "square_feet" :1, #conversion en m² et passage des 0 à NaN => inutile que 4% des cas renseignés
"price" : 1, #conversion en nombre et passage des 0 à NaN
#"weekly_price" : 1,#conversion en nombre et passage des 0 à NaN vérifier si complement des NaN Price
#"monthly_price" : 1,#conversion en nombre et passage des 0 à NaN vérifier si complement des NaN Price
"security_deposit" : 1,#conversion en nombre et passage des 0 à NaN
#"cleaning_fee" : 1,#conversion en nombre et passage des 0 à NaN
#    "guests_included" : 1, #???
#"extra_people" : 1,#conversion en nombre et passage des 0 à NaN
#    "minimum_nights" : 1, #difficile à interpréter
#    "maximum_nights" : 1,#difficile à interpréter
"availability_30" : 1, #découpage en quantile
"availability_60" : 1,#découpage en quantile
"availability_90" : 1,#découpage en quantile
"availability_365" : 1,#découpage en quantile
"number_of_reviews" : 1, #nombre de commentaire => on garde les 0
"first_review" : 1, #date à transformer en nb de mois pour déterminer une ancienneté?
"last_review" : 1, #date à transformer en nb de mois pour déterminer une ancienneté?
"review_scores_rating" : 1, #notes
"review_scores_accuracy" : 1,#notes
"review_scores_cleanliness" : 1,#notes
"review_scores_checkin" : 1,#notes
"review_scores_communication" : 1,#notes
"review_scores_location" : 1,#notes
"review_scores_value" : 1,#notes
#    "requires_license" : 1, #toujours à False
#    "jurisdiction_names" : 1, #toujours Paris
"instant_bookable" : 1,#True/False
#"cancellation_policy" : 1, #regroupement des faibles modalités
#"require_guest_profile_picture" : 1, #True/False
#"require_guest_phone_verification" : 1,#True/False
"calculated_host_listings_count" : 1, #permet de voir les multi-détentions dans l'échantillon
"reviews_per_month":1 #nombre de commentaires moyen par mois
})
airbnb = DataFrame(list(rec))

In [10]:
####################################
# Recodage et typage des variables #
####################################
#pour transformer des zéro en NaN
def to_nan(x):
    val = 0
    if x==0:
        val = numpy.NaN
    else:
        val = x
    return val

#pour transformer les chaîne de caractère montant en $ en nombre
def dollar_to_number (x) : 
    val = 0 
    if x==None:
        val = None
    else :
        val = float(x.replace('$','').replace(",",""))
    return val

airbnb["price"]=airbnb["price"].apply(dollar_to_number).apply(to_nan)
airbnb["security_deposit"]=airbnb["security_deposit"].apply(dollar_to_number).apply(to_nan)

#Pour transformer des % en nombre
def percent_to_number (x) : 
    val = 0 
    if x==None:
        val = None
    else :
        val = int(x.replace('%',''))
    return val
airbnb["host_response_rate"]=airbnb["host_response_rate"].apply(percent_to_number)
airbnb["zipcode"]=pandas.to_numeric(airbnb["zipcode"], errors='coerce').fillna(0).astype(int)
airbnb["zipcode"]=airbnb["zipcode"].astype(int)


In [11]:
airbnb["name"]=airbnb["name"].str.replace('%', '')



In [12]:
####################################
# Reclassement des variables et    #
# découpages en 3 tables           #
# APPART, HOST puis REVIEWS        #
####################################
airbnb_appart=airbnb[['_id',
'name',
'neighbourhood_cleansed',
'zipcode',
'latitude',
'longitude' ,
'property_type' ,
'room_type',
'accommodates',
'bathrooms',
'bedrooms',
'beds',
'price',
'security_deposit',
'availability_30',
'availability_60' ,
'availability_90',
'availability_365',
'instant_bookable',
'host_id',
]]
airbnb_appart.rename(columns={'_id':'home_id'}, inplace=True)
airbnb_host=airbnb[[
'host_id',
'host_name' ,
'host_since',
'host_location',
'host_response_time',
'host_response_rate',
'host_is_superhost' ,
'host_neighbourhood',
'host_listings_count',
'calculated_host_listings_count',
]]
airbnb_host=airbnb_host.drop_duplicates()
airbnb_group_reviews=airbnb[[
'_id',
'host_id',
'number_of_reviews' ,
'first_review',
'last_review',
'review_scores_rating',
'review_scores_accuracy',
'review_scores_cleanliness',
'review_scores_checkin',
'review_scores_communication',
'review_scores_location',
'review_scores_value',
'reviews_per_month']]
airbnb_group_reviews.rename(columns={'_id':'home_id'}, inplace=True)


/home/user/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


In [13]:
####################################
# Export en Json                   #
####################################

airbnb_appart.to_json('airbnb_appart_echantillon.json',orient='records',force_ascii = False)
airbnb_host.to_json('airbnb_host_echantillon.json',orient='records',force_ascii = False)
airbnb_group_reviews.to_json('airbnb_group_reviews_echantillon.json',orient='records',force_ascii = False)

In [15]:
####################################
# Chargement des commentaires      #
####################################

rec = db.reviews.find({"listing_id" : {'$lte':5000} },
{    "_id" : 1, 
    "listing_id" : 1, 
    "date" : 1, 
    "reviewer_id" : 1, 
    "reviewer_name" :1})
airbnb_reviews = DataFrame(list(rec))
airbnb_reviews = airbnb_reviews[[ 
    "_id" , 
    "listing_id" , 
    "date", 
    "reviewer_id", 
    "reviewer_name"]]
airbnb_reviews.rename(columns={'_id':'review_id', 'listing_id':'home_id'}, inplace=True)

airbnb_reviews.to_json('airbnb_reviews_echantillon.json',orient='records',force_ascii = False)